In [1]:
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import sqlite3

In [2]:
#connect to database
path = "model/db/HeartDiseaseByCounty.db"
cnx = sqlite3.connect(path)

In [3]:
#Display tables from provisional database
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", cnx)

,name
0,State
1,Heart_Disease_Mortality_State_Counties
2,Population_State_Counties
3,Features_State_Counties


In [4]:
#Read all tables from provisional database into DataFrames
df_features = pd.read_sql_query("SELECT * FROM Features_State_Counties", cnx)
df_features.head()

,State_Name,County,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,...,Population_Asian,Population_NHPI,Population_Hispanic,Population_NHW,Percent_Insufficient_Sleep,Average_Traffic_Volume_per_Meter_of_Major_Roadways,Percent_Homeowners,Percent_Severe_Housing_Cost_Burden,Percent_Children_in_Poverty,Teen_Birth_Rate
0,Alabama,Autauga,21,4.7,4.7,18,33,7.2,35,69,...,681,62,1649,41316,36,88,75,13.0,19.0,25.0
1,Alabama,Baldwin,18,4.2,4.3,17,31,8.0,27,74,...,2508,146,10131,181201,33,87,74,12.0,14.0,28.0
2,Alabama,Barbour,30,5.4,5.2,22,42,5.6,24,53,...,113,46,1064,11356,39,102,61,14.0,44.0,41.0
3,Alabama,Bibb,19,4.6,4.6,19,38,7.8,34,16,...,53,26,588,16708,38,29,75,10.0,28.0,42.0
4,Alabama,Blount,22,4.9,4.9,19,34,8.4,30,16,...,185,70,5536,50255,36,33,79,8.0,18.0,34.0


In [5]:
df_state = pd.read_sql_query("SELECT * FROM State", cnx)
df_state.head()

,Name,Code
0,Alaska,AK
1,Alabama,AL
2,Arkansas,AR
3,Arizona,AZ
4,California,CA


In [6]:
df_pop = pd.read_sql_query("SELECT * FROM Population_state_counties", cnx)
df_pop.head()

,State_Code,County,PopEst
0,AK,Aleutians East,"3,337"
1,AK,Aleutians West,"5,634"
2,AK,Anchorage,"288,000"
3,AK,Bethel,"18,386"
4,AK,Bristol Bay,836


In [7]:
df_mortality = pd.read_sql_query("SELECT * FROM Heart_disease_mortality_state_counties", cnx)
df_mortality.head()

,State_Code,County,Rate,Level
0,AK,Aleutians East,165.000000,1
1,AK,Aleutians West,261.800000,1
2,AK,Anchorage,261.733333,1
3,AK,Bethel,321.322222,2
4,AK,Bristol Bay,0.000000,1


In [8]:
#Pre-process data - standardize and convert all State names into State codes
statename = df_state["Name"].values.tolist()


In [9]:
symbol = df_state["Code"].values.tolist()


In [10]:
#Pre-process data - transform State_Name into codes for later merging 
df_features.replace(to_replace = statename, value = symbol, inplace = True)
df_features.tail()

,State_Name,County,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,...,Population_Asian,Population_NHPI,Population_Hispanic,Population_NHW,Percent_Insufficient_Sleep,Average_Traffic_Volume_per_Meter_of_Major_Roadways,Percent_Homeowners,Percent_Severe_Housing_Cost_Burden,Percent_Children_in_Poverty,Teen_Birth_Rate
3137,WY,Sweetwater,15,3.4,3.6,18,30,7.7,25,90,...,435,68,6924,34145,31,155,75,NaN,NaN,NaN
3138,WY,Teton,12,3.0,3.2,15,12,8.2,12,100,...,326,35,3434,18812,27,135,58,NaN,NaN,NaN
3139,WY,Uinta,16,3.6,3.7,17,36,7.4,27,84,...,96,30,1875,17741,31,96,75,NaN,NaN,NaN
3140,WY,Washakie,16,3.6,3.7,17,29,8.3,28,83,...,65,8,1108,6498,28,83,77,NaN,NaN,NaN
3141,WY,Weston,14,3.5,3.7,17,33,7.9,27,63,...,135,2,284,6267,30,64,78,NaN,NaN,NaN


In [11]:
#Merge first two DataFrames together
df_combined1 = df_features.merge(df_mortality, left_on = ["State_Name","County"], right_on = ["State_Code","County"])
df_combined1.head()

,State_Name,County,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,...,Population_NHW,Percent_Insufficient_Sleep,Average_Traffic_Volume_per_Meter_of_Major_Roadways,Percent_Homeowners,Percent_Severe_Housing_Cost_Burden,Percent_Children_in_Poverty,Teen_Birth_Rate,State_Code,Rate,Level
0,AL,Autauga,21,4.7,4.7,18,33,7.2,35,69,...,41316,36,88,75,13.0,19.0,25.0,AL,422.022222,4
1,AL,Baldwin,18,4.2,4.3,17,31,8.0,27,74,...,181201,33,87,74,12.0,14.0,28.0,AL,321.570588,2
2,AL,Barbour,30,5.4,5.2,22,42,5.6,24,53,...,11356,39,102,61,14.0,44.0,41.0,AL,461.144444,4
3,AL,Bibb,19,4.6,4.6,19,38,7.8,34,16,...,16708,38,29,75,10.0,28.0,42.0,AL,393.036364,3
4,AL,Blount,22,4.9,4.9,19,34,8.4,30,16,...,50255,36,33,79,8.0,18.0,34.0,AL,387.481818,3


In [12]:
df_combined1.shape

(3013, 52)

In [13]:
#Pre-process data - standardize County name and convert PopEst into integers
df_pop['County'] = df_pop["County"].str.replace(' County', "")
df_pop["PopEst"] = pd.to_numeric(df_pop["PopEst"].str.replace(',',''))
df_pop.head()

,State_Code,County,PopEst
0,AK,Aleutians East,3337.0
1,AK,Aleutians West,5634.0
2,AK,Anchorage,288000.0
3,AK,Bethel,18386.0
4,AK,Bristol Bay,NaN


In [14]:
df_pop.dropna(inplace = True)
df_pop.isnull().sum

<bound method DataFrame.sum of       State_Code  County  PopEst
0          False   False   False
1          False   False   False
2          False   False   False
3          False   False   False
5          False   False   False
...          ...     ...     ...
3137       False   False   False
3138       False   False   False
3139       False   False   False
3140       False   False   False
3141       False   False   False

[3105 rows x 3 columns]>

In [15]:
#Merge all DataFrames together 

df_combined = df_combined1.merge(df_pop, how = "inner", left_on = ["State_Name","County"], right_on = ["State_Code","County"])
df_combined.head()

,State_Name,County,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,...,Average_Traffic_Volume_per_Meter_of_Major_Roadways,Percent_Homeowners,Percent_Severe_Housing_Cost_Burden,Percent_Children_in_Poverty,Teen_Birth_Rate,State_Code_x,Rate,Level,State_Code_y,PopEst
0,AL,Autauga,21,4.7,4.7,18,33,7.2,35,69,...,88,75,13.0,19.0,25.0,AL,422.022222,4,AL,55869.0
1,AL,Baldwin,18,4.2,4.3,17,31,8.0,27,74,...,87,74,12.0,14.0,28.0,AL,321.570588,2,AL,223234.0
2,AL,Barbour,30,5.4,5.2,22,42,5.6,24,53,...,102,61,14.0,44.0,41.0,AL,461.144444,4,AL,24686.0
3,AL,Bibb,19,4.6,4.6,19,38,7.8,34,16,...,29,75,10.0,28.0,42.0,AL,393.036364,3,AL,22394.0
4,AL,Blount,22,4.9,4.9,19,34,8.4,30,16,...,33,79,8.0,18.0,34.0,AL,387.481818,3,AL,57826.0


In [16]:
# Drop State and county columns as they are identifier, and remove Level (classification target),
#Remove "Rate" (from which we derive mortality classes)

X = df_combined.copy()
X = X.drop([ "State_Name","County", "Rate","Level", "State_Code_x", "State_Code_y"], 1)
X = X.fillna(0)
X.head()

,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,Percent_Excessive_Drinking,Percent_Uninsured,...,Population_NHPI,Population_Hispanic,Population_NHW,Percent_Insufficient_Sleep,Average_Traffic_Volume_per_Meter_of_Major_Roadways,Percent_Homeowners,Percent_Severe_Housing_Cost_Burden,Percent_Children_in_Poverty,Teen_Birth_Rate,PopEst
0,21,4.7,4.7,18,33,7.2,35,69,15,9,...,62,1649,41316,36,88,75,13.0,19.0,25.0,55869.0
1,18,4.2,4.3,17,31,8.0,27,74,18,11,...,146,10131,181201,33,87,74,12.0,14.0,28.0,223234.0
2,30,5.4,5.2,22,42,5.6,24,53,13,12,...,46,1064,11356,39,102,61,14.0,44.0,41.0,24686.0
3,19,4.6,4.6,19,38,7.8,34,16,16,10,...,26,588,16708,38,29,75,10.0,28.0,42.0,22394.0
4,22,4.9,4.9,19,34,8.4,30,16,14,13,...,70,5536,50255,36,33,79,8.0,18.0,34.0,57826.0


In [17]:
#Target: Classification of mortality risks 
y = df_combined["Level"].ravel()
y[:5]

array([4, 2, 4, 3, 3], dtype=int64)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [19]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [21]:
rf_model = rf_model.fit(X_train_scaled, y_train)

In [22]:
predictions = rf_model.predict(X_test_scaled)

In [23]:
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 1", "Actual 2", "Actual 3", "Actual 4"], columns=["Predicted 1", "Predicted 2", "Predicted 3", "Predicted 4"])

cm_df

,Predicted 1,Predicted 2,Predicted 3,Predicted 4
Actual 1,99,72,1,2
Actual 2,28,143,24,13
Actual 3,5,50,45,58
Actual 4,4,17,28,145


In [24]:
acc_score = accuracy_score(y_test, predictions)
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

,Predicted 1,Predicted 2,Predicted 3,Predicted 4
Actual 1,99,72,1,2
Actual 2,28,143,24,13
Actual 3,5,50,45,58
Actual 4,4,17,28,145


Accuracy Score : 0.5885558583106267
Classification Report
              precision    recall  f1-score   support

           1       0.73      0.57      0.64       174
           2       0.51      0.69      0.58       208
           3       0.46      0.28      0.35       158
           4       0.67      0.75      0.70       194

    accuracy                           0.59       734
   macro avg       0.59      0.57      0.57       734
weighted avg       0.59      0.59      0.58       734



In [25]:
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.04702523392253512, 'Median_Household_Income'),
 (0.036660675306290906, 'Polution_Average_Daily_PM2.5'),
 (0.03580422345554223, 'Average_Number_of_Mentally_Unhealthy_Days'),
 (0.0356907743078979, 'Percent_Smokers'),
 (0.03503685226070159, 'Household_Income_White'),
 (0.03486490633113343, 'Percent_Fair_or_Poor_Health'),
 (0.03377366387555132, 'Average_Number_of_Physically_Unhealthy_Days'),
 (0.03146550473305125, 'Percent_Physically_Inactive'),
 (0.030208971727716017, 'Population_Hispanic'),
 (0.025977103417311652, 'Population_Asian'),
 (0.025961096153320266, 'Preventable_Hospitalization_Rate'),
 (0.022871837087735696, 'Population_NHPI'),
 (0.022853371932235366, 'Percent_Some_College'),
 (0.02212619291605177, 'Population_Black'),
 (0.021982406528113188, 'Percent_Drive_Alone_to_Work'),
 (0.02185229994412147, 'Population_AIAN'),
 (0.02094378413977611, 'Social_Association_Rate'),
 (0.020635889354100424, 'Average_Traffic_Volume_per_Meter_of_Major_Roadways'),
 (0.020583987980151822, 'Popul